### Info
based on notebook4

compare qkeras and qonnx predictions (without using hls4ml)\
strange thing on github: pred_qonnx[0]: [[4.4136963]]; then: 4.4375

### Load packages and the model

In [ ]:
import tensorflow as tf
import numpy as np
import h5py
from qkeras.utils import _add_supported_quantized_objects
from qkeras import quantized_bits
co = {}; _add_supported_quantized_objects(co)

model_path = f"/eos/project/c/cms-l1ml/public/Pawel/axol1tl_model.h5"

model = tf.keras.models.load_model(model_path, custom_objects=co)

model.summary()

In [ ]:
model.get_config()['layers'][1]['config']['activation']['class_name']

In [ ]:
for layer in model.get_config()['layers'][1:9]:
    print(layer['class_name'],'\t',layer['config']['activation'])

### Copy weights

In [ ]:
ds_weights = data_file['model']['trimmed_encoder']['model_weights']
for layer in iter(model.layers):
    #print(layer.name)
    if layer.name in ('input_1','subtract','dot'): continue

    if layer.name == 'q_dense_4':
        weights_group = ds_weights['q_dense_4']['variational_auto_encoder']['vae__encoder']['q_dense_4']
        layer.set_weights((weights_group['kernel:0'], weights_group['bias:0']))
        continue

    weights_group = ds_weights[layer.name][layer.name]
    layer.set_weights((weights_group['kernel:0'], weights_group['bias:0']))

### QONNX model

In [1]:
#Quick setup (load qonnx model)
import tensorflow as tf
import numpy as np
import h5py
from qkeras.utils import _add_supported_quantized_objects
from qkeras import quantized_bits

import qonnx.core.onnx_exec as oxe
from qonnx.converters import from_keras
from qonnx.core.modelwrapper import ModelWrapper

qonnx_model = ModelWrapper("mod_model.onnx")

2025-10-07 21:57:43.837811: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-10-07 21:57:43.872521: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-10-07 21:57:43.872558: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-10-07 21:57:43.872599: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-10-07 21:57:43.880995: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-10-07 21:57:43.881499: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

In [2]:
node_names = []
for node in qonnx_model.graph.node:
    node_names.append(node.name)

print(node_names)

['Quant_0', 'Quant_1', 'Quant_2', 'Quant_3', 'Quant_4', 'Quant_5', 'Quant_6', 'Quant_7', 'Quant_8', 'Quant_9', 'Quant_10', 'Quant_11', 'Quant_12', 'Quant_13', 'Quant_14', 'Quant_15', 'MatMul_0', 'Add_0', 'Quant_16', 'Relu_0', 'Quant_17', 'MatMul_1', 'Add_1', 'Quant_18', 'Relu_1', 'Quant_19', 'MatMul_2', 'Add_2', 'Quant_20', 'Relu_2', 'Quant_21', 'MatMul_3', 'Add_3', 'Quant_22', 'Relu_3', 'Quant_23', 'MatMul_4', 'Add_4', 'Quant_24', 'MatMul_5', 'Add_5', 'Quant_25', 'Relu_4', 'Quant_26', 'MatMul_6', 'Add_6', 'Quant_27', 'Relu_5', 'Quant_28', 'MatMul_7', 'Add_7', 'Quant_29', 'Sub_0', 'Quant_30', 'Quant_31', 'Unsqueeze_0', 'Unsqueeze_1', 'MatMul_8', 'Quant_32', 'Squeeze_0', 'Identity_0', 'Identity_1']


### Check the input parameters of the Quant nodes (adjust the inputs and attributes)
0.125 = (0.001)_2 \
0.0625 = (0.0001)_2 \
0.0009765625 = (0.0000000001)_2

#### Inputs

In [7]:
qnodes = list(filter(lambda x: x.op_type == "Quant", qonnx_model.graph.node))

# qnode consumers and param inputs
for qnode in qnodes[30:]:
    print("Quant node name: ",qnode.name)
    print("Consumer name: ",qonnx_model.find_consumer(qnode.output[0]).name)
    for inp in qnode.input[1:]:
        print(inp, qonnx_model.get_initializer(inp))
    print("--------------------")
# Only the Quant_19 node has two consumers ('MatMul_2' and later also 'Sub_0')

Quant node name:  Quant_30
Consumer name:  Unsqueeze_1
Quant_30_param0 0.0625
Quant_30_param1 0.0
Quant_30_param2 18.0
--------------------
Quant node name:  Quant_31
Consumer name:  Unsqueeze_0
Quant_31_param0 0.0625
Quant_31_param1 0.0
Quant_31_param2 18.0
--------------------
Quant node name:  Quant_32
Consumer name:  Squeeze_0
Quant_32_param0 0.0625
Quant_32_param1 0.0
Quant_32_param2 18.0
--------------------


In [ ]:
qnodes_before_relu = list(filter(lambda x: qonnx_model.find_consumer(x.output[0]).name in ["Relu_" + str(i) for i in range(0,8)], qnodes))
qnodes_before_relu

In [ ]:
# 18 bits width (as in accumulator precision)
for node in qnodes_before_relu:
    qonnx_model.set_initializer(node.name+"_param2", np.array(18))

In [ ]:
# 10 bits to the right from the dot
for node in qnodes_before_relu:
    qonnx_model.set_initializer(node.name+"_param0",np.array(0.0009765625))

In [6]:
# correct precision of last layers
rel_nodes = ['Quant_3' + str(i) for i in np.arange(0,3)]
print(rel_nodes)

for node_name in rel_nodes:
    qonnx_model.set_initializer(node_name+"_param0",np.array(0.0625))

['Quant_30', 'Quant_31', 'Quant_32']


#### Attributes
Attributes alright

In [ ]:
# qnode attributes
for qnode in qnodes[32:]:
    print("Quant node name: ",qnode.name)
    print(qnode.attribute)
    print("---------------")

In [ ]:
from qonnx.custom_op.base import CustomOp
from qonnx.custom_op.registry import getCustomOp

In [ ]:
# Signed values before relu
for node_ in qnodes_before_relu:
    node_op = getCustomOp(qonnx_model.graph.node[qonnx_model.get_node_index(node_)])
    node_op.set_nodeattr("signed",1)

### Final nodes

In [ ]:
qonnx_model.get_node_from_name('Squeeze_0')

Unsqueeze_0 gives shape [1,10,1] \
Unsqueeze_1 gives shape [1,1,10] \
Both take the Sub_0_out0 \
MatMul_8 multiplies (order of inputs): Unsqueeze_1_out0 * Unsqueeze_0_out0

In [ ]:
qonnx_model.get_initializer('Squeeze_0_param0')

In [ ]:
qonnx_model.get_tensor_valueinfo('MatMul_8_out0')

### Conclusions
In hls input_1 (global input) should have precision fixed<14,7> (now fixed<18,8>)\
Relu should take input with precision like the accumulator: fixed<18,8>\
Should check the "signed" attribute of Quant nodes\
missing quantizers before 'Sub_0' and later on

Quant node is not present between each add and relu within the Dense layers.\
These quant nodes should correspond to accumulators (?) \
Check inputs of the Subtract node \
Strange "Linear" layers in the hls_config (the same as in qkeras)

### Add missing Quant nodes

In [ ]:
from qonnx.transformation.quantize_graph import QuantizeGraph

qnode_map = {"name": {"MatMul_8": [(("output",0), (scale_,zeropoint_,bitwidth_,narrow_,signed_,rounding_mode_))]}}
qnode_map

In [ ]:
scale_ = 0.0009765625
zeropoint_ = 0
bitwidth_ = 18
narrow_ = 0
signed_ = 1
rounding_mode_ = "ROUND"

In [ ]:
quantize_graph = QuantizeGraph(qnode_map)
(qonnx_model,_) = quantize_graph.apply(qonnx_model)

In [8]:
qonnx_model.save("mod_model.onnx")